In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns

In [2]:
pd.__version__

'1.1.3'

### 1. Iris

In [3]:
iris = sns.load_dataset('iris')
iris.groupby(iris.species).describe()

sepal_length                                              \
                  count   mean       std  min    25%  50%  75%  max   
species                                                               
setosa             50.0  5.006  0.352490  4.3  4.800  5.0  5.2  5.8   
versicolor         50.0  5.936  0.516171  4.9  5.600  5.9  6.3  7.0   
virginica          50.0  6.588  0.635880  4.9  6.225  6.5  6.9  7.9   

           sepal_width         ... petal_length      petal_width         \
                 count   mean  ...          75%  max       count   mean   
species                        ...                                        
setosa            50.0  3.428  ...        1.575  1.9        50.0  0.246   
versicolor        50.0  2.770  ...        4.600  5.1        50.0  1.326   
virginica         50.0  2.974  ...        5.875  6.9        50.0  2.026   

                                               
                 std  min  25%  50%  75%  max  
species                                        
setosa      0.105386  0.1  0.2  0.2  0.3  0.6  
versicolor  0.197753  1.0  1.2  1.3  1.5  1.8  
virginica   0.274650  1.4  1.8  2.0  2.3  2.5  

[3 rows x 32 columns]

In [4]:
def get_new_stat(x):
    Q1 = x.quantile(.25)
    Q3 = x.quantile(.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    higher = Q3 + 1.5 * IQR
    outlier = False
    new = []
    for e in x:
        if e > higher or e < lower:
            outlier = True
            new.append(np.nan)
        else:
            new.append(e)
    s = pd.Series(new)
    return outlier, s.mean(), s.std()

In [5]:
s_sl = iris[iris.species == 'setosa']['sepal_length']
type(s_sl)

pandas.core.series.Series

In [6]:
species = []; items = []
outlier_list = []; mean_list = []; std_list = []
for spec in ['setosa', 'versicolor', 'virginica']:
    for wl in ['sepal_length','sepal_width','petal_length','petal_width']:
        s = iris[iris.species == spec][wl]
        outlier, new_mean, new_std = get_new_stat(s)
        #print(f'{spec} {wl}: {outlier}, {new_mean.round(3)}, {new_std.round(3)}')
        species.append(spec); items.append(wl)
        outlier_list.append(outlier)
        mean_list.append(new_mean); std_list.append(new_std)

In [7]:
tmp_iris = pd.DataFrame({
    'species': species,
    'items': items,
    'outlier': outlier_list,
    'new_mean': mean_list,
    'new_std': std_list
})
new_iris = tmp_iris.set_index(['species', 'items'])
new_iris

outlier  new_mean   new_std
species    items                                    
setosa     sepal_length    False  5.006000  0.352490
           sepal_width      True  3.431250  0.320343
           petal_length     True  1.460870  0.129062
           petal_width      True  0.233333  0.085883
versicolor sepal_length    False  5.936000  0.516171
           sepal_width     False  2.770000  0.313798
           petal_length     True  4.285714  0.437798
           petal_width     False  1.326000  0.197753
virginica  sepal_length     True  6.622449  0.593459
           sepal_width      True  2.955319  0.260275
           petal_length    False  5.552000  0.551895
           petal_width     False  2.026000  0.274650

In [8]:
old_mean_list = []; old_std_list = []
for spec in ['setosa', 'versicolor', 'virginica']:
    for wl in ['sepal_length','sepal_width','petal_length','petal_width']:
        s = iris[iris.species == spec][wl]
        #print(f'{spec} {wl}: {s.mean().round(3)}, {s.std().round(3)}')
        old_mean_list.append(s.mean())
        old_std_list.append(s.std())

In [9]:
new_iris['old_mean'] = old_mean_list
new_iris['old_std'] = old_std_list
new_iris

outlier  new_mean   new_std  old_mean   old_std
species    items                                                        
setosa     sepal_length    False  5.006000  0.352490     5.006  0.352490
           sepal_width      True  3.431250  0.320343     3.428  0.379064
           petal_length     True  1.460870  0.129062     1.462  0.173664
           petal_width      True  0.233333  0.085883     0.246  0.105386
versicolor sepal_length    False  5.936000  0.516171     5.936  0.516171
           sepal_width     False  2.770000  0.313798     2.770  0.313798
           petal_length     True  4.285714  0.437798     4.260  0.469911
           petal_width     False  1.326000  0.197753     1.326  0.197753
virginica  sepal_length     True  6.622449  0.593459     6.588  0.635880
           sepal_width      True  2.955319  0.260275     2.974  0.322497
           petal_length    False  5.552000  0.551895     5.552  0.551895
           petal_width     False  2.026000  0.274650     2.026  0.274650

### 2. Titanic

In [11]:
titanic = sns.load_dataset('titanic')

In [12]:
# a
titanic['category1'] = titanic.apply(lambda r: r.sex if r.age >= 20 else 'child', axis=1)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,NaN,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


In [13]:
# b
titanic['age'] = titanic['age'].fillna(titanic['age'].mean().round(1))
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,29.7,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,child
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


In [14]:
# a
titanic['category1'] = titanic.apply(lambda r: r.sex if r.age >= 20 else 'child', axis=1)
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,category1
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,male
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,29.7,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,female
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,male
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,male


In [21]:
# c
def alive_ratio(x):
    return x.yes / (x.no + x.yes)
titanic.groupby([titanic.sex]).agg(alive_ratio)

AttributeError: 'DataFrame' object has no attribute 'yes'

In [20]:
titanic.groupby([titanic.sex])